In [ ]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import MinMaxScaler, VectorAssembler
import numpy as np

In [ ]:
def Kmeans(data,k,ct=0.0001,iterations=30, initial_centroids=None):
 # Step 1: Normalize the data using MinMaxScaler
    feature_columns = data.columns[:-1]  # Exclude the target column
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
    data_with_features = assembler.transform(data)

    scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
    scaler_model = scaler.fit(data_with_features)
    normalized_data = scaler_model.transform(data_with_features).select("scaled_features")

    # Convert normalized data to RDD for map-reduce operations
    data_rdd = normalized_data.rdd.map(lambda row: row[0].toArray())

    # Step 2: Initialize centroids
    if initial_centroids is None:
        centroids = data_rdd.takeSample(False, k)
    else:
        centroids = initial_centroids

    for iteration in range(iterations):
        # Step 3: Assign points to the nearest centroid
        clusters = data_rdd.map(lambda point: (closest_centroid(point, centroids), point))

        # Step 4: Update centroids by averaging points in each cluster
        new_centroids = clusters.groupByKey().mapValues(average_points).collect()
        new_centroids = sorted(new_centroids, key=lambda x: x[0])  # Sort by cluster index
        new_centroids = [centroid[1] for centroid in new_centroids]  # Extract centroid positions

        # Step 5: Check for convergence
        if has_converged(centroids, new_centroids, ct):
            break

        centroids = new_centroids

    return [tuple(np.round(c, 5)) for c in centroids]

def closest_centroid(point, centroids):
    distances = [np.linalg.norm(np.array(point) - np.array(c)) for c in centroids]
    return np.argmin(distances)

def average_points(points):
    points = list(points)
    return np.mean(points, axis=0)

def has_converged(old_centroids, new_centroids, threshold):
    total_movement = sum(np.linalg.norm(np.array(o) - np.array(n)) for o, n in zip(old_centroids, new_centroids))
    return total_movement < threshold



In [ ]:
# """ ******************* Importing the data ******************* """



Iris = "/content/Iris.csv"
file_type = "csv"
infer_schema = "True"
first_row_is_header = "True"
delimiter = ","
iris = SparkSession.builder.appName("KMeansExample").getOrCreate().read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(Iris)

# """ ******************* Example for testing ******************* """


new_centroids = Kmeans(iris, 2, initial_centroids=[(0.5,0.5,0.5,0.5,0.5),(0.3,0.3,0.3,0.3,0.3)])
round_new_centroids=[tuple(round(num, 5) for num in tup) for tup in new_centroids]
expected_new_centroids=[(0.16871,0.19553,0.58252,0.08475,0.06618),(0.67067,0.54882,0.36532,0.66478,0.65951)]
if (not len(new_centroids)==len(expected_new_centroids)):
    print("Test 1 Failed - Number of clusters is different than requested")
if set(round_new_centroids)==set(expected_new_centroids):
    print("Test 1 passed successfully")
else:
    print("Test 1 failed")


Test 1 passed successfully
